Requirements.txt

torch 2.0.0+cu117
transformers 4.28.1
numpy
typing
tqdm
gc
pandas
collections
sklearn
contextlib
io
IPython
os

In [1]:
%load_ext autoreload
%autoreload 2


import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,2,3,4"
import sys
import torch
import pandas as pd
from torch import nn
import numpy as np
from torch.optim.lr_scheduler import ExponentialLR
import wandb

sys.path.append("../NLP-DL-Project-hypo-to-hyper/pipeline_src/")


from config.config import TaskConfig
from train import CustomScheduler, train
from logger.logger import WanDBWriter
from trainer.train_epoch import train_epoch, predict
from metrics.metrics import get_all_metrics
from dataset.dataset import init_data
from logger.logger import WanDBWriter


if torch.cuda.is_available():
    device = "cuda"
    print("GPU")
else:
    device = "cpu"
    print("CPU")


SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
print(torch.cuda.device_count())

GPU
4


In [2]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    AutoConfig,
    AutoModelForCausalLM,
    LlamaTokenizer,
    LlamaForCausalLM,
)

from peft import LoraConfig, get_peft_model, get_peft_model_state_dict


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('FILE')}
  warn(msg)
/srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')

In [3]:
config = TaskConfig()

config.n_epochs = 4
config.batch_size = 2
config.lr = 3e-4
config.min_lr = 3e-6

config.validation = 1
config.save_every = 1
config.compute_metrics_every = 1

config.data_path = 'babel_datasets/train_en_babel.pickle'
config.gold_path = (
    None  # "SemEval2018-Task9/training/gold/1A.english.training.gold.txt"
)
config.test_data_path = 'babel_datasets/test_en_babel.pickle'
config.test_gold_path = (
    None  # "SemEval2018-Task9/test/gold/1A.english.test.gold.txt"
)

config.device = device
config.using_peft = True
config.model_type = "Llama"  # Auto or Llama
config.wandb_log_dir = "/raid/rabikov/wandb/"
config.model_checkpoint = "eachadea/vicuna-13b-1.1"
config.exp_name = config.model_checkpoint.replace("/", "-")
config.saving_path = "/raid/rabikov/model_checkpoints/" + config.exp_name

In [4]:
tokenizer = LlamaTokenizer.from_pretrained(
    config.model_checkpoint,
    padding_side="left",
)
train_dataset, test_dataset, train_loader, val_loader = init_data(tokenizer, config)

In [8]:
train_dataset[0]

{'encoded_term': tensor([    1, 21099,   919,  7498,  1112,   962, 29879,   363,   278,  1734,
           525, 26204, 29889, 29876, 29889, 29896,  4286, 29871,   673, 29901]),
 'encoded_target': tensor([    1,  5225, 29889, 29876, 29889, 29906, 29892,  2211, 29889, 29876,
         29889, 29896, 29892,  3023, 29889, 29876, 29889, 29896, 29892,  5320,
         29889, 29876, 29889, 29896, 29892,  4832, 29889, 29876, 29889, 29896,
         29892,  9881, 29889, 29876, 29889, 29896, 29892,  9475, 29889, 29876,
         29889, 29896, 29892, 14183, 29889, 29876, 29889, 29896, 29892,  7581,
         29918, 26204, 29889, 29876, 29889, 29896, 29892, 13677, 29918, 26204,
         29889, 29876, 29889, 29896, 29892,   868,   356, 29883,  3039, 29918,
         26204, 29889, 29876, 29889, 29896, 29892, 15090,  1943, 29883,  3039,
         29918, 26204, 29889, 29876, 29889, 29896, 29892,  4725,   284, 29918,
         26204, 29889, 29876, 29889, 29896, 29892,  7282, 29918, 26204, 29889,
         29876, 

In [18]:
max_len = 0
k = 0
for i in range(len(train_dataset)):
    try:
        len_cur = len(train_dataset[i]['input_seq'])
        if len_cur > 2048:
            k += 1
            max_len = len_cur
            max_idx = i
    except TypeError:
        print(i)

In [19]:
k

19

# grand - parent - child
# parent1, parent2 - child

In [14]:
max_len, max_idx

(5930, 12331)

In [16]:
len(train_dataset.data[max_idx]['children'])

470

In [17]:
train_dataset.data[max_idx]

{'children': ['Communist.n.1',
  'demagogue.n.1',
  'Democrat.n.1',
  'Federalist.n.1',
  'Potemkin.n.1',
  'machine_politician.n.1',
  'Labourite.n.1',
  'Mugwump.n.1',
  'noncandidate.n.1',
  'party_man.n.1',
  'Republican.n.1',
  'sachem.n.1',
  'standard-bearer.n.1',
  'technocrat.n.2',
  'Whig.n.3',
  'James_Fiennes,_1st_Baron_Saye_and_Sele.n.1',
  'Scott_LeDoux.n.1',
  'Morys_Bruce,_4th_Baron_Aberdare.n.1',
  'Hildred_Carlile.n.1',
  'Mark_R._Bacon.n.1',
  'Juan_Pablo_Pérez_Alfonzo.n.1',
  'Thomas_Walker_Gilmer.n.1',
  'Basil_Glass.n.1',
  'Bob_Cooper_(politician).n.1',
  'Harold_T._Johnson.n.1',
  "Pat_O'Rawe.n.1",
  'Campbell_Cavasso.n.1',
  'Romeo_Munoz_Cachola.n.1',
  'Joaquim_António_de_Aguiar.n.1',
  'Sir_Harry_Burrard,_1st_Baronet,_of_Walhampton.n.1',
  'Charles_Oaks_Ermatinger.n.1',
  'Herman_Badillo.n.1',
  'Elias_Weber_Bingeman_Snider.n.1',
  'William_J._Twaddell.n.1',
  'Sylvester_Neelon.n.1',
  'James_Norris_(politician).n.1',
  'Ralph_Neville-Grenville.n.1',
  'Airey

In [4]:
if config.model_type == "Auto":
    model_type = AutoModelForCausalLM
    tokenizer_type = AutoTokenizer
elif config.model_type == "Llama":
    model_type = LlamaForCausalLM
    tokenizer_type = LlamaTokenizer

model = model_type.from_pretrained(
    config.model_checkpoint,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
if config.using_peft:
    LORA_R = 8
    LORA_ALPHA = 16
    LORA_DROPOUT = 0.05
    LORA_TARGET_MODULES = [
        "q",
        "v",
    ]

    # model = prepare_model_for_int8_training(model)
    config_lora = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        # target_modules=LORA_TARGET_MODULES,
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config_lora)
    model.print_trainable_parameters()

trainable params: 6553600 || all params: 13022417920 || trainable%: 0.05032552357220002


In [6]:
train_dataset, test_dataset, train_loader, val_loader = init_data(tokenizer, config)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
    model.parameters(), lr=config.lr, betas=(0.9, 0.98), eps=1e-9
)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=config.lr,
#                                                  steps_per_epoch=len(train_loader), epochs=config.n_epochs)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=len(train_loader) * config.n_epochs, eta_min=config.min_lr
)

In [8]:
# wandb
logger = WanDBWriter(config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vityavitalich. Use `wandb login --relogin` to force relogin


In [9]:
train(
    model,
    tokenizer,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    criterion,
    logger,
    config,
)

Start of the epoch 0


eval going: 100%|██████████| 750/750 [20:23<00:00,  1.63s/it]


Start of the epoch 1


eval going: 100%|██████████| 750/750 [20:23<00:00,  1.63s/it]


Start of the epoch 2


eval going: 100%|██████████| 750/750 [20:15<00:00,  1.62s/it]


Start of the epoch 3


eval going: 100%|██████████| 750/750 [20:10<00:00,  1.61s/it]
